In [1]:
import sys
sys.path.append(r'../')

In [2]:
from dolfin import *

import numpy as np

import cell_geom as geom
import cell_material as mat
import cell_computation as comp

## Linear Backend

In [3]:
parameters['linear_algebra_backend'] = 'Eigen'

## Define Geometry

In [5]:
mesh = Mesh(r'../m.xml')

In [6]:
cell = geom.UnitCell(mesh)

In [7]:
# Add inclusion
inc = geom.InclusionCircle(2, (0.5, 0.5), 0.25)
inc_di = {'circle_inc': inc}
cell.set_append_inclusion(inc_di)

## Define Material

In [8]:
E_m, nu_m, E_i, nu_i = 10.0, 0.3, 1000.0, 0.3
mat_m = mat.st_venant_kirchhoff(E_m, nu_m)
mat_i = mat.st_venant_kirchhoff(E_i, nu_i)
mat_li = [mat_m, mat_i]

## Define Computation

In [9]:
VFS = VectorFunctionSpace(cell.mesh, "CG", 1, 
                          constrained_domain=geom.PeriodicBoundary_no_corner(2))

In [10]:
def deform_grad_with_macro(F_bar, w_component):
    return F_bar + grad(w_component)

In [11]:
w = Function(VFS)
strain_space = TensorFunctionSpace(mesh, 'DG', 0)
compute = comp.MicroComputation(cell, mat_li, 
                                [deform_grad_with_macro],
                                [strain_space])

In [12]:
F_bar = [0.9, 0., 0., 1.]

In [13]:
compute.input([F_bar], [w])

In [14]:
# comp.set_solver_parameters('non_lin_newton', lin_method='direct',
#                       linear_solver='cholesky')

In [15]:
compute.comp_fluctuation(print_progress=True, print_solver_info=False)

fluctuation computation finished


In [16]:
compute.view_fluctuation()

In [17]:
delta = 0.01

In [18]:
for i in range(10):
    F_bar[0] -= delta
    print F_bar
    compute.input([F_bar], [w])
    compute.comp_fluctuation(print_progress=True, print_solver_info=False)

[0.89, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.88, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.87, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.86, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.85, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.84, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.83, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.82, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.8099999999999999, 0.0, 0.0, 1.0]
fluctuation computation finished
[0.7999999999999999, 0.0, 0.0, 1.0]
fluctuation computation finished
